In [ ]:
pip install --upgrade torch torchvision

In [ ]:
# https://drive.google.com/file/d/1S7upK1OsxQoLfOLDTR-Px3XJFGuZKn3B/view?usp=sharing
!gdown https://drive.google.com/uc?id=1S7upK1OsxQoLfOLDTR-Px3XJFGuZKn3B

In [ ]:
!rm -r TLDataset/ALL/PKG-C-NMC2019/train/.ipynb_checkpoints
!rm -r TLDataset/ALL/PKG-C-NMC2019/train/.DS_STORE

In [ ]:
from torchvision.models import ConvNeXt_Base_Weights

In [ ]:
!pip install seaborn

In [ ]:
pip install --upgrade efficientnet-pytorch

In [ ]:
!pip install torch-summary

In [ ]:
!pip install summary

# Training

In [1]:
# start training
!python train-effnet.py --model_name efficientnet_b0 --data_dir TLDataset/ALL/PKG-C-NMC2019 --num_classes 2 --num_epochs 50 --batch_size 64 --learning_rate 0.001 --checkpoint_path model_checkpoints/ALL/effnet/

Using device: cuda:0
Training Data: 7845 samples
Validation Data: 2347 samples
model.safetensors: 100%|████████████████████| 21.4M/21.4M [00:00<00:00, 218MB/s]
Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 32, 112, 112]        (864)
├─BatchNormAct2d: 1-2                         [-1, 32, 112, 112]        --
|    └─Identity: 2-1                          [-1, 32, 112, 112]        --
|    └─SiLU: 2-2                              [-1, 32, 112, 112]        --
├─Sequential: 1-3                             [-1, 320, 7, 7]           --
|    └─Sequential: 2-3                        [-1, 16, 112, 112]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 16, 112, 112]        (1,448)
|    └─Sequential: 2-4                        [-1, 24, 56, 56]          --
|    |    └─InvertedResidual: 3-2             [-1, 24, 56, 56]          (6,004)
|    |    └─InvertedResidual: 3-3             [-1, 24, 56, 56]          

# Evaluation 

ALL Only Evaluation

In [2]:
!python evaluate-effnet.py --model_name efficientnet_b0 --data_dir ALL_Unseen/ --model_path model_checkpoints/ALL/effnet/efficientnet_b0_best.pth --num_classes 2 --batch_size 64

Evaluating: 100%|██████████████████████| 8/8 [00:01<00:00,  6.62it/s, acc=0.829]

Evaluation Results:
Precision: 0.8033
Recall: 0.8033
F1: 0.8033
Accuracy: 0.8294
Balanced accuracy: 0.8033


# Inference

In [ ]:
!python inference.py --model_name resnet50 --image_path TLDataset/ALL/PKG-C-NMC2019/C-NMC_test_prelim_phase_data --model_path model_checkpoints/ALL/resnet50_best_epoch_49.pth --num_classes 2 --image_save results_C-NMC_test_prelim_phase_data

In [ ]:
!python inference.py --model_name resnet50 --image_path TLDataset/ALL/PKG-C-NMC2019/C-NMC_test_final_phase_data --model_path model_checkpoints/ALL/resnet50_best_epoch_49.pth --num_classes 2 --image_save results_C-NMC_test_final_phase_data

# Transfer learning 

Here the trained model on ALL dataset is finetuned for AML dataset, by loading the ALL model weights as pretrained model.

In [3]:
!python train-effnet.py --model_name efficientnet_b0 --data_dir AMLDataset/AML --num_classes 2 --num_epochs 50 --batch_size 64 --learning_rate 0.001 --checkpoint_path model_checkpoints/AML/effnet --pretrained_path model_checkpoints/ALL/effnet/efficientnet_b0_latest.pth

Using device: cuda:0
Training Data: 12207 samples
Validation Data: 5230 samples
Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 32, 112, 112]        (864)
├─BatchNormAct2d: 1-2                         [-1, 32, 112, 112]        --
|    └─Identity: 2-1                          [-1, 32, 112, 112]        --
|    └─SiLU: 2-2                              [-1, 32, 112, 112]        --
├─Sequential: 1-3                             [-1, 320, 7, 7]           --
|    └─Sequential: 2-3                        [-1, 16, 112, 112]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 16, 112, 112]        (1,448)
|    └─Sequential: 2-4                        [-1, 24, 56, 56]          --
|    |    └─InvertedResidual: 3-2             [-1, 24, 56, 56]          (6,004)
|    |    └─InvertedResidual: 3-3             [-1, 24, 56, 56]          (10,710)
|    └─Sequential: 2-5                        [-1, 40, 28, 28]         

# Notes

Inference on unseen AML Images

In [ ]:
# TL model with unseen ALL images again to see if TL helped

In [4]:
!python evaluate-effnet.py --model_name efficientnet_b0 --data_dir ALL_Unseen/ --model_path model_checkpoints/AML/effnet/efficientnet_b0_best.pth --num_classes 2 --batch_size 64

Evaluating: 100%|██████████████████████| 8/8 [00:01<00:00,  6.67it/s, acc=0.608]

Evaluation Results:
Precision: 0.4098
Recall: 0.4597
F1: 0.4153
Accuracy: 0.6077
Balanced accuracy: 0.4597


In [ ]:
# TL model with unseen AML images

In [5]:
!python evaluate-effnet.py --model_name efficientnet_b0 --data_dir AML_Unseen/ --model_path model_checkpoints/AML/effnet/efficientnet_b0_latest.pth --num_classes 2 --batch_size 64

Evaluating: 100%|████████████████████| 15/15 [00:01<00:00,  9.73it/s, acc=0.916]

Evaluation Results:
Precision: 0.8816
Recall: 0.8435
F1: 0.8607
Accuracy: 0.9160
Balanced accuracy: 0.8435


In [ ]:
# Training AML Only without TF weights to Compare with TF Model

In [ ]:
!pip install seaborn

In [6]:
# start training AML Model
!python train-effnet.py --model_name efficientnet_b0 --data_dir AMLDataset/AML/ --num_classes 2 --num_epochs 50 --batch_size 64 --learning_rate 0.001 --checkpoint_path model_checkpoints/AML_Only/effnet/

Using device: cuda:0
Training Data: 12207 samples
Validation Data: 5230 samples
Layer (type:depth-idx)                        Output Shape              Param #
├─Conv2d: 1-1                                 [-1, 32, 112, 112]        (864)
├─BatchNormAct2d: 1-2                         [-1, 32, 112, 112]        --
|    └─Identity: 2-1                          [-1, 32, 112, 112]        --
|    └─SiLU: 2-2                              [-1, 32, 112, 112]        --
├─Sequential: 1-3                             [-1, 320, 7, 7]           --
|    └─Sequential: 2-3                        [-1, 16, 112, 112]        --
|    |    └─DepthwiseSeparableConv: 3-1       [-1, 16, 112, 112]        (1,448)
|    └─Sequential: 2-4                        [-1, 24, 56, 56]          --
|    |    └─InvertedResidual: 3-2             [-1, 24, 56, 56]          (6,004)
|    |    └─InvertedResidual: 3-3             [-1, 24, 56, 56]          (10,710)
|    └─Sequential: 2-5                        [-1, 40, 28, 28]         

In [ ]:
# Test AML_Only model on AML Unseen Data

In [7]:
!python evaluate-effnet.py --model_name efficientnet_b0 --data_dir AML_Unseen/ --model_path model_checkpoints/AML_Only/effnet/efficientnet_b0_latest.pth --num_classes 2 --batch_size 64

Evaluating: 100%|████████████████████| 15/15 [00:01<00:00,  9.97it/s, acc=0.916]

Evaluation Results:
Precision: 0.8957
Recall: 0.8269
F1: 0.8553
Accuracy: 0.9160
Balanced accuracy: 0.8269


In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
#-------- **************-----------------***********---

In [ ]:
# Best AML model weights to TF on ALL data

In [ ]:
!python train-mobilenetv2.py --model_name mobilenetv2 --data_dir TLDataset/ALL/PKG-C-NMC2019 --num_classes 2 --num_epochs 150 --batch_size 64 --learning_rate 0.01 --momentum 0.9 --checkpoint_path model_checkpoints/AML_Pretrained/mobilenetv2 --pretrained_path model_checkpoints/AML_Only/mobilenetv2/mobilenetv2_best_epoch_104.pth

In [ ]:
# TF Model Validation on Unseen ALL

In [ ]:
!python evaluate-mobilenetv2.py --model_name mobilenetv2 --data_dir ALL_Unseen/ --model_path model_checkpoints/AML_Pretrained/mobilenetv2/mobilenetv2_best_epoch_100.pth --num_classes 2 --batch_size 32

In [ ]:
# TF Model Validation on Unseen AML

In [ ]:
!python evaluate.py --model_name resnet50 --data_dir AML_Unseen/ --model_path model_checkpoints/ALL_Pretrained/resnet50_best_epoch_76.pth --num_classes 2 --batch_size 32